In [9]:
import os
import json
from openai import OpenAI
from pydantic import BaseModel, Field, ValidationError
from typing import List, Dict, Any
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
# --- 1. Define the Structured Output (The Plan) using Pydantic ---
# This ensures the LLM's output is predictable and easy to work with.

class ToolCall(BaseModel):
    """A single tool call to be executed by a specialized agent."""
    tool_name: str = Field(..., description="The exact name of the tool to be called.")
    parameters: Dict[str, Any] = Field({}, description="The parameters to pass to the tool.")
    reasoning: str = Field(..., description="A brief explanation of why this tool was chosen.")

class Plan(BaseModel):
    """The complete, step-by-step plan generated by the Core Brain."""
    thought: str = Field(..., description="A high-level summary of the plan and reasoning.")
    workflow_type: str = Field(..., description="Either 'sequential' or 'parallel'.", enum=["sequential", "parallel"])
    steps: List[ToolCall] = Field(..., description="The sequence of tool calls to execute.")


# --- 2. Create the Core Brain Class ---

class CoreBrain:
    """The central LLM orchestrator that creates plans."""
    def __init__(self):
        # Initialize the OpenAI client. It will automatically find the API key
        # from the environment variables.
        self.client = OpenAI()
        if not self.client.api_key:
            raise ValueError("OPENAI_API_KEY environment variable not found.")

    def create_plan(self, scenario: str) -> Plan:
        """
        Takes a natural language scenario and returns a structured plan.
        """
        # This is the most important part: the system prompt.
        # It guides the LLM on its role, capabilities, and expected output format.
        system_prompt = f"""
        You are the 'Planning Agent' for Synapse, an AI-powered last-mile coordinator for Grab.
        Your primary role is to analyze a disruption scenario and generate a structured, step-by-step action plan in JSON format.
        
        You have access to the following tools:
        - notify_customer(message: str, user_id: str): Informs the customer of a situation.
        - reroute_driver(driver_id: str, new_task_id: str): Assigns a driver to a short, nearby delivery.
        - get_nearby_merchants(cuisine_type: str, max_wait_time: int): Finds similar restaurants with shorter wait times.
        - initiate_mediation_flow(case_id: str, user_id: str, driver_id: str): Starts a real-time resolution process.
        
        Based on the user's scenario, you must:
        1.  Think through a logical plan to resolve the issue.
        2.  Decide if the steps should be run 'sequential' or 'parallel'.
        3.  Create a list of tool calls to execute.
        4.  Provide reasoning for each step.
        5.  Respond ONLY with a valid JSON object that conforms to the following schema.

        JSON Schema:
        {json.dumps(Plan.model_json_schema(), indent=2)}
        """

        try:
            print("🧠 Core Brain is thinking...")
            response = self.client.chat.completions.create(
                model="gpt-4o", # Or "gpt-3.5-turbo" for faster, cheaper prototyping
                response_format={"type": "json_object"},
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"Scenario: {scenario}"}
                ]
            )
            
            response_json = json.loads(response.choices[0].message.content)
            print("✅ LLM generated a raw plan.")

            # --- Validate the output against our Pydantic model ---
            validated_plan = Plan(**response_json)
            print("✅ Plan validated successfully.")
            return validated_plan

        except ValidationError as e:
            print(f"❌ Validation Error: The LLM returned an invalid plan format. {e}")
            return None
        except Exception as e:
            print(f"❌ An unexpected error occurred: {e}")
            return None

# --- 3. Example Usage ---

if __name__ == "__main__":
    # Instantiate the brain
    brain = CoreBrain()
    
    # Define a disruption scenario based on your documentation
    disruption_scenario = """
    An order was placed for user 'user_123' at 'Burger Palace'.
    However, the get_merchant_status() tool detected a 40-minute kitchen prep time.
    The assigned driver is 'driver_456'.
    """
    
    # Generate a plan
    generated_plan = brain.create_plan(disruption_scenario)
    
    if generated_plan:
        print("\n--- 🤖 Generated Plan ---")
        print(f"Thought: {generated_plan.thought}")
        print(f"Workflow Type: {generated_plan.workflow_type}")
        print("Steps:")
        for i, step in enumerate(generated_plan.steps, 1):
            print(f"  {i}. Tool: {step.tool_name}")
            print(f"     Params: {step.parameters}")
            print(f"     Reasoning: {step.reasoning}")
        print("------------------------\n")

🧠 Core Brain is thinking...
❌ An unexpected error occurred: Error code: 401 - {'error': {'message': 'Incorrect API key provided: key-here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
